In [1]:
from retrieval_base.retrieval_setup import RetrievalSetup

# Import input file as 'conf'
conf_string = 'config_fiducial_K_A_new'
conf = __import__(conf_string, fromlist=[''])

RS = RetrievalSetup(conf)

species v0.9.0

Working folder: /net/lem/data1/regt/retrieval_base/Luhman_16

Configuration file: /net/lem/data1/regt/retrieval_base/Luhman_16/species_config.ini
Database file: /net/lem/data1/regt/retrieval_base/Luhman_16/species_database.hdf5
Data folder: /net/lem/data1/regt/retrieval_base/Luhman_16/data

Configuration settings:
   - Database: /net/lem/data1/regt/retrieval_base/Luhman_16/species_database.hdf5
   - Data folder: /net/lem/data1/regt/retrieval_base/Luhman_16/data
   - Magnitude of Vega: 0.03

Multiprocessing: mpi4py installed
Process number 1 out of 1...


/net/lem/data1/regt/conda_env/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


In [2]:
RS.ParamTable.table

,name,m_set,Param,val,idx_free
0,log_P_range,all,None,"(-5, 3)",None
1,n_atm_layers,all,None,50,None
2,lbl_opacity_sampling,all,None,3,None
3,parallax,all,None,496,None
4,inclination,all,None,18,None
5,do_scat_emis,all,None,False,None
6,log_a,all,<retrieval_base.parameters_new.Parameter objec...,NaN,0
7,a,all,None,NaN,None
8,log_l,all,<retrieval_base.parameters_new.Parameter objec...,NaN,1
9,l,all,None,NaN,None


In [4]:
import numpy as np
cube = np.random.uniform(0, 1, 26)
phys_cube = RS.ParamTable(cube)
RS.ParamTable.table

,name,m_set,Param,val,idx_free
0,log_P_range,all,None,"(-5, 3)",None
1,n_atm_layers,all,None,50,None
2,lbl_opacity_sampling,all,None,3,None
3,parallax,all,None,496,None
4,inclination,all,None,18,None
5,do_scat_emis,all,None,False,None
6,log_a,all,<retrieval_base.parameters_new.Parameter objec...,-0.193073,0
7,a,all,None,0.641103,None
8,log_l,all,<retrieval_base.parameters_new.Parameter objec...,-2.716533,1
9,l,all,None,0.001921,None


In [8]:
RS.ParamTable.get('opa_base_gray_0')

3.436388033505548e-08

In [ ]:
from parameters_new import ParameterTable

In [ ]:
constant_params = {
    'const_1': 1., 
    'const_2': 2., 
    'A': {'const_3': 3.},  
    }
free_params = {
    'free_4': ('U', (0,6), r'free_a'), 
    'free_5': ('G', (5,1), r'free_b'), 
    'A': {
        'free_6': ('U', (5,7), r'free_6')
        },
    'B': {
        'free_7': ('U', (4,7), r'free_7'), 
        'log_free_8': ('U', (2,3), r'log_free_8')
        },
    }

PI = ParameterTable(
    free_params, constant_params, 
    model_settings=['A','B'], 
    PT_kwargs={
        'PT_mode':'free_gradient', 
        'A':{'n_T_knots':5, 'PT_mode':'grid'}, 
        #'A':{'n_T_knots':5}
        }, 
    )

PI(cube=[0.5, 0.4, 0.5, 0.5, 0.5], ndim=5)
PI.table

In [ ]:
PI.PT_kwargs

In [ ]:
cloud_kwargs = {
    #'cloud_mode': 'gray', 
    'wave_cloud_0': 2.0, 
    'A': {'wave_cloud_0': 2.5}, 
    'B': {'cloud_mode': 'EddySed'}, 
    }
new_cloud_kwargs = PI.expand_per_model_setting(cloud_kwargs, main_m_set='all')
print(new_cloud_kwargs)

new_cloud_kwargs['A'].update(new_cloud_kwargs['all'])
print(new_cloud_kwargs)

In [ ]:
PI.queried_m_set = 'B'; print(PI.get('free_8'))
PI.queried_m_set = ''; print(PI.get('free_8'))

PI.queried_m_set = 'all'; print(PI.get('free_5'))